In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

demandareal = pd.read_excel("setembro2021.xlsx")
demandareal = demandareal["CONSUMO"].values

# Parâmetros
volume_maximo = 5
volume_minimo = 2
vazao = 2
probcruz = 0.6
probmut = 0.02
num_individuals = 150


def fitness(individuo, demanda, volume_maximo, volume_minimo, custo, vazao, check=0):
    volume = volume_maximo

    # Encontra os índices dos períodos em que a válvula está aberta
    idx = np.where(individuo == 1)[0]

    # Calcula o fitness como a soma dos custos dos períodos abertos
    fit = np.sum(custo[idx])
    
   # Calcula a diferença entre os valores do indivíduo
    diff = np.diff(individuo)

    # Adiciona um elemento zero no início do vetor diff
    diff = np.insert(diff, 0, 0)

    # Calcula o volume em cada período
    volume_t = diff * vazao + demanda
    volume_t = np.array([volume] + list(volume_t))
    volume_t = np.cumsum(volume_t)

    # Limita os valores do volume entre o mínimo e o máximo
    volume_t = np.clip(volume_t, volume_minimo, volume_maximo)

    # Calcula a penalidade por violar as restrições de volume
    volume_penal = np.abs(volume_t - volume_maximo) + np.abs(volume_t - volume_minimo)

    # Soma a penalidade ao fitness
    fit += np.sum(volume_penal)

    if check == 1:
        print(fit, len(volume_t))

    return fit

def S(t):
    hora = t % 24  # Calcula a hora do dia a partir do índice t
    if hora >= 17 and hora < 21:  # Horário de ponta
        return 1.63527
    else:  # Horário fora de ponta
        return 0.62124
        
tarifa = np.array([S(i) for i in range(len(demandareal))])

def selecao(populacao, aptidao):
    # Embaralha a população
    populacao = np.random.permutation(populacao)
    
    # Escolhe dois indivíduos para o torneio binário
    a = populacao[0]
    b = populacao[1]
    
    # Calcula as aptidões dos indivíduos
    apt_a = fitness(a, demandareal, volume_maximo, volume_minimo, tarifa, vazao)
    apt_b = fitness(b, demandareal, volume_maximo, volume_minimo, tarifa, vazao)
    
    # Retorna o melhor indivíduo e sua aptidão
    if apt_a < apt_b:
        return a, apt_a
    else:
        return b, apt_b

def cruzamento(pais, probcruz):
    if np.random.rand() < probcruz:
        # Gera uma máscara aleatória de bits
        mask = np.random.randint(0, 2, size=len(pais[0]))
        
        # Aplica a operação xor entre os pais e a máscara
        filho = np.bitwise_xor(pais[0], mask)
        filho = np.bitwise_xor(filho, pais[1])
        
        # Converte os bits para booleanos
        filho = filho.astype(bool)
        
    else:
        # Escolhe um dos pais aleatoriamente
        filho = pais[np.random.randint(0, 2)]
        
        # Converte o valor booleano em um array de um elemento
        filho = np.array([filho])
    
    return filho


def mutacao(individuo, probmut):
    n = len(individuo)
    
    # Gera um vetor binomial com a probabilidade de mutação
    mut = np.random.binomial(1, probmut, size=n)
    
    # Aplica a operação xor entre o indivíduo e o vetor de mutação
    copia = np.bitwise_xor(individuo, mut)
    
    # Converte os bits para booleanos
    copia = copia.astype(bool)

    return copia

def AG(demandareal, tarifa, volume_maximo, volume_minimo, vazao, probcruz, probmut, num_individuals):
    
    np.random.seed(424)
    
    num_genes = len(demandareal)
    
    # Cria uma população inicial de indivíduos binários
    populacao = np.random.randint(0, 2, size=(num_individuals, num_genes))
    
    # Converte os bits para booleanos
    populacao = populacao.astype(bool)
    
    F = lambda individuo: fitness(individuo, demandareal, volume_maximo, volume_minimo, tarifa, vazao)
    
    # Calcula a aptidão de cada indivíduo na população
    aptidao = np.array([F(individuo) for individuo in populacao])

    historico = []
    
    contador = 0

    while True:

        # Encontra os índices do melhor e do pior indivíduo na população
        min_idx = np.argmin(aptidao)
        max_idx = np.argmax(aptidao)


        # Seleciona dois pais para o cruzamento
        pais, apt = selecao(populacao, aptidao)

        # Realiza o cruzamento entre os pais
        filho = cruzamento(pais, probcruz)
        
        # Realiza a mutação no filho
        filho = mutacao(filho, probmut)

        # Substitui o pior indivíduo pelo filho
        populacao[max_idx] = filho
        
        # Atualiza a aptidão do pior indivíduo
        aptidao[max_idx] = F(filho)

        # Adiciona o valor da aptidão mínima e média ao histórico
        historico.append([aptidao[min_idx], np.mean(aptidao)])

        # Verifica o critério de parada
        if (np.mean(aptidao) - aptidao[min_idx] < 1e-5) or (aptidao[min_idx] < 6000):
            break
        
        contador += 1
        
        if (contador % 1000 == 0):
            print(aptidao[min_idx])
    
    # Encontra o melhor indivíduo na população final
    melhor_solucao = populacao[min_idx]
    
    num_1s = np.sum(melhor_solucao)
    num_0s = len(melhor_solucao) - num_1s
    
    print("Número de 1s:", num_1s)
    print("Número de 0s:", num_0s)

    historico = np.array(historico)
    
    return historico, populacao[min_idx]

dados, melhor_valor = AG(demandareal, tarifa, volume_maximo, volume_minimo, vazao, probcruz, probmut, num_individuals)

resultado = fitness(melhor_valor, demandareal, volume_maximo, volume_minimo, tarifa, vazao)

print("Resultado do algoritmo genético:", resultado)


Número de 1s: 320
Número de 0s: 400
2413.51233 721
Resultado do algoritmo genético: 2413.51233


In [15]:
fitness(melhor_valor, demandareal, volume_maximo, volume_minimo, tarifa, vazao, check=1)



2413.51233 721


2413.51233